# Test normalizations
## basic settings and imports first

In [1]:
import libsmoother

from bokeh.plotting import figure
from bokeh.palettes import viridis
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column

import json
import sys
import pickle

output_notebook()

KBP = 1000
MBP = 1000 * KBP

WINDOW_SIZES = [1 * MBP, 5 * MBP, 10 * MBP]
BIN_SIZES = [50 * KBP, 100 * KBP, 500 * KBP]
OUTPUT_BACKEND = "svg"
if True: # fast
    WINDOW_SIZES = [1 * MBP]
    BIN_SIZES = [100 * KBP]
    OUTPUT_BACKEND = "canvas"
NUM_SAMPLES = [4**x for x in range(8)]

CHECK_THAT_HEATMAPS_ARE_EXACTLY_THE_SAME = False

Loading BokehJS ...

## define basic evaluation functions

In [2]:
import json
import sys

def lib_sps_print(s):
    pass

def conf_quarry_basic(quarry):
    #warnings.filterwarnings('ignore')
    with libsmoother.open_default_json() as default_file:
        default_json = json.load(default_file)
        quarry.set_value(["settings"], default_json)
    quarry.set_value(["settings", "filters", "cut_off_bin"], "smaller")
    quarry.set_value(["settings", "filters", "show_contig_smaller_than_bin"], True)
    quarry.set_value(["settings", "interface", "fixed_bin_size"], True)
    quarry.set_value(["settings", "interface", "add_draw_area", "val"], 0)
    #quarry.set_value(["settings", "normalization", "ice_sparse_slice_filter", "val"], 0)


def conf_quarry_data(quarry):
    quarry.set_value(["settings", "normalization", "scale"], "dont")
    quarry.set_value(["settings", "normalization", "log_base", "val"], 0)

def conf_quarry_heatmap(quarry):
    quarry.set_value(["settings", "normalization", "scale"], "max")
    quarry.set_value(["settings", "normalization", "log_base", "val"], 10)


def set_bin_size(quarry, bin_size):
    div = quarry.get_value(["dividend"])
    if bin_size % div != 0:
        print("WARNING: uneven division by index dividend", file=sys.stderr)
    if bin_size < div:
        print("WARNING: dividend larger than value", file=sys.stderr)
    bin_size = max(1, bin_size // div)
    quarry.set_value(["settings", "interface", "fixed_bin_size_x", "val"], bin_size)
    quarry.set_value(["settings", "interface", "fixed_bin_size_y", "val"], bin_size)

def tsv_to_ret(data, key, tsv):
    ret = [(x[:-1], x[-1]) for x in tsv]
    ret.sort()
    if CHECK_THAT_HEATMAPS_ARE_EXACTLY_THE_SAME:
        data[key + ("bin_coords", )] = [a for a, _ in ret]
    data[key + ("bin_vals", )] = [b for _, b in ret]

def quarry_whole_window(data, key, quarry):
    canvas_size_x, canvas_size_y = quarry.get_canvas_size(lib_sps_print)
    quarry.set_value(["area", "x_start"], 0)
    quarry.set_value(["area", "x_end"], canvas_size_x)
    quarry.set_value(["area", "y_start"], 0)
    quarry.set_value(["area", "y_end"], canvas_size_y)
    
    conf_quarry_data(quarry)
    tsv_to_ret(data, key, quarry.get_heatmap_export(lib_sps_print))
    
    conf_quarry_heatmap(quarry)
    data[key + ("heatmap", )] = quarry.get_heatmap(lib_sps_print)

def quarry_chunked_window(data, data_key, quarry, window_size):
    canvas_size_x, canvas_size_y = quarry.get_canvas_size(lambda s: None)
    tsv = []
    heatmap = None
    for x_start in range(0, canvas_size_x, window_size):
        quarry.set_value(["area", "x_start"], x_start)
        quarry.set_value(["area", "x_end"], x_start + window_size)
        for y_start in range(0, canvas_size_y, window_size):
            quarry.set_value(["area", "y_start"], y_start)
            quarry.set_value(["area", "y_end"], y_start + window_size)
            conf_quarry_data(quarry)
            tsv.extend(quarry.get_heatmap_export(lib_sps_print))
            
            conf_quarry_heatmap(quarry)
            heatmap_local = quarry.get_heatmap(lib_sps_print)
            if heatmap is None:
                heatmap = heatmap_local
            else:
                for key, val in heatmap_local.items():
                    heatmap[key].extend(val)
    tsv_to_ret(data, data_key, tsv)
    data[data_key + ("heatmap", )] = heatmap

## Load index and compute data

In [3]:
quarry = libsmoother.Quarry("../../smoother_out/radicl.smoother_index")
conf_quarry_basic(quarry)

data = {}

for bin_size in BIN_SIZES:
    print("bin_size", bin_size // KBP, "k")

    set_bin_size(quarry, bin_size)
    print("grid-seq")
    quarry.set_value(["settings", "filters", "symmetry"], "all")
    quarry.set_value(["settings", "normalization", "normalize_by"], "grid-seq")
    # max out num samples for the default setting
    quarry.set_value(
        ["settings", "normalization", "grid_seq_samples", "val"],
        quarry.get_value(["contigs", "genome_size"])
    )
    quarry_whole_window(data, ("grid-seq", "GT", bin_size), quarry)

    #print("ddd")
    #quarry.set_value(["settings", "normalization", "normalize_by"], "dont")
    #quarry.set_value(["settings", "normalization", "ddd"], True)
    ## max out num samples for the default setting
    #quarry.set_value(
    #    ["settings", "normalization", "ddd_samples", "val_max"],
    #    quarry.get_value(["contigs", "genome_size"]),
    #)
    #quarry_whole_window(data, ("ddd", "GT", bin_size), quarry)
    #quarry.set_value(["settings", "normalization", "ddd"], False)

    print("cooler")
    quarry.set_value(["settings", "filters", "symmetry"], "mirror")
    quarry.set_value(["settings", "normalization", "normalize_by"], "cool-ice")
    quarry_whole_window(data, ("ICE", "cooler", bin_size), quarry)

    print("raw data")
    quarry.set_value(["settings", "normalization", "normalize_by"], "dont")
    quarry_whole_window(data, ("raw-mirror", bin_size), quarry)

    quarry.set_value(["settings", "filters", "symmetry"], "all")
    quarry.set_value(["settings", "normalization", "normalize_by"], "dont")
    quarry_whole_window(data, ("raw-all", bin_size), quarry)

    for window_size in WINDOW_SIZES:
        print("window_size", window_size // KBP, "k")

        quarry.set_value(["settings", "normalization", "ice_local"], False)
        for num_samples in NUM_SAMPLES:
            print("grid-seq", num_samples, "samples")
            quarry.set_value(["settings", "filters", "symmetry"], "all")
            quarry.set_value(["settings", "normalization", "normalize_by"], "grid-seq")
            quarry.set_value(["settings", "normalization", "grid_seq_samples", "val"], num_samples)
            quarry_chunked_window(data, ("grid-seq", "sampled", bin_size, window_size, num_samples), 
                                  quarry, window_size)

            #print("ddd", num_samples, "samples")
            #quarry.set_value(["settings", "normalization", "normalize_by"], "dont")
            #quarry.set_value(["settings", "normalization", "ddd_samples", "val_max"], num_samples)
            #quarry.set_value(["settings", "normalization", "ddd"], True)
            #quarry_chunked_window(data, ("ddd", "sampled", bin_size, window_size, num_samples), 
            #                      quarry, window_size)
            #quarry.set_value(["settings", "normalization", "ddd"], False)

            print("ice", num_samples, "samples")
            quarry.set_value(["settings", "normalization", "normalize_by"], "ice")
            quarry.set_value(["settings", "filters", "symmetry"], "mirror")
            quarry.set_value(["settings", "normalization", "num_ice_bins", "val"], num_samples)
            
            quarry_chunked_window(data, ("ICE", "global", bin_size, window_size, num_samples), 
                                  quarry, window_size)
        if CHECK_THAT_HEATMAPS_ARE_EXACTLY_THE_SAME:
            assert data[("ICE", "cooler", bin_size, "bin_coords")] == \
                data[("ICE", "global", bin_size, window_size, num_samples, "bin_coords")]

bin_size 100 k
grid-seq
cooler
raw data
window_size 1000 k
grid-seq 1 samples
ice 1 samples
grid-seq 4 samples
ice 4 samples
grid-seq 16 samples
ice 16 samples
grid-seq 64 samples
ice 64 samples
grid-seq 256 samples
ice 256 samples
grid-seq 1024 samples
ice 1024 samples
grid-seq 4096 samples
ice 4096 samples
grid-seq 16384 samples
ice 16384 samples


## Investigate the scatter plot for one bin and window size

Expect a bad correleation for a low number of samples, it should then gradually improve with the number of samples.

In [5]:
import random
from bokeh.models import Legend

def get_mean_dev(ground_truth, points):
    return sum(abs(a-b) for a, b in zip(ground_truth, points)) / len(points)


def plot_scatter_points(ground_truth, data, title):
    ALMOST_ZERO = 10**-5
    palette = viridis(sum(1 if c is None else 0 for _1, _2, c in data))
    f = figure(
            title=title, 
            x_axis_type="log", 
            y_axis_type="log", 
            x_range=(ALMOST_ZERO, 10**5), 
            y_range=(ALMOST_ZERO, 10**5),
            sizing_mode="stretch_width",
        )
    f.line(x=[ALMOST_ZERO,1], y=[ALMOST_ZERO,1], color="black")
    items = []
    idx = 0
    for name, points, color in data:
        xs = []
        ys = []
        for (x, y) in random.sample(list(zip(ground_truth, points)), min(len(points), 1000)):
            xs.append(x)
            ys.append(y)
        mean_dev = round(get_mean_dev(ground_truth, points), 5)
        d = f.dot(x=xs, y=ys, color=palette[idx] if color is None else color, size=25, alpha=0.5)
        idx += 1 if color is None else 0
        items.append((name + " dev: " + str(mean_dev), [d]))

    f.xaxis.axis_label = "ground truth"
    f.yaxis.axis_label = "sample"
    f.add_layout(Legend(items=items, location="center", click_policy="hide"), "right")
    f.output_backend = OUTPUT_BACKEND
    show(f)

plot_scatter_points(
    data[("ICE", "cooler", BIN_SIZES[0], "bin_vals")], 
    [("unnormalized", data[("raw-mirror", BIN_SIZES[0], "bin_vals")], "red")] +
    [("num samples = " + str(num_samples), 
      data[("ICE", "global", BIN_SIZES[0], WINDOW_SIZES[0], num_samples, "bin_vals")], None) for num_samples in NUM_SAMPLES[:-2]], 
    "icing; bin-size= " + str(BIN_SIZES[0] // KBP) + "k window-size= " + str(WINDOW_SIZES[0]//KBP) + "k" )

plot_scatter_points(
    data[("grid-seq", "GT", BIN_SIZES[0], "bin_vals")], 
    [("unnormalized", data[("raw-all", BIN_SIZES[0], "bin_vals")], "red")] +
    [("num samples = " + str(num_samples), 
      data[("grid-seq", "sampled", BIN_SIZES[0], WINDOW_SIZES[0], num_samples, "bin_vals")], None) for num_samples in NUM_SAMPLES[:-3]], 
    "grid-seq; bin-size= " + str(BIN_SIZES[0] // KBP) + "k window-size= " + str(WINDOW_SIZES[0]//KBP) + "k" )

#plot_scatter_points(
#    data[("ddd", "GT", BIN_SIZES[0], "bin_vals")], 
#    [("unnormalized", data[("raw-all", BIN_SIZES[0], "bin_vals")], "red")] +
#    [("num samples = " + str(num_samples), 
#      data[("ddd", "sampled", BIN_SIZES[0], WINDOW_SIZES[0], num_samples, "bin_vals")], None) for num_samples in NUM_SAMPLES[:-3]], 
#    "Distance Dependant Decay; bin-size= " + str(BIN_SIZES[0] // KBP) + "k window-size= " + str(WINDOW_SIZES[0]//KBP) + "k" )

mean deviation becomes smaller with increasing number of samples adn approaches the origin diagonal

## Plot mean deviation as a function of the number of samples for all bin and window sizes

In [6]:
def corr_as_func_of_samples(conditions):
    palette = viridis(len(conditions))
    f = figure(
            y_axis_type="log", 
            x_axis_type="log"
        )
    idx = 0
    for ground_truth, sample, name in conditions:
        xs = []
        ys = []
        for num_samples, points in sample:
            xs.append(num_samples)
            ys.append(get_mean_dev(ground_truth, points))
        f.line(x=xs, y=ys, color=palette[idx], legend_label=name)
        f.x(x=xs, y=ys, color=palette[idx], legend_label=name)
        idx += 1
    f.xaxis.axis_label = "number of samples"
    f.yaxis.axis_label = "mean deviation"
    f.legend.click_policy="hide"
    f.output_backend = OUTPUT_BACKEND
    show(f)

conditions_ice = [
    (
        data[("ICE", "cooler", bin_size, "bin_vals")], 
        [(num_samples, data[("ICE", "global", bin_size, window_size, num_samples, "bin_vals")]) for num_samples in NUM_SAMPLES], 
        "ICE bin_size=" + str(bin_size//KBP) + "k window_size=" + str(window_size//KBP) + "k"
     ) 
     for bin_size in BIN_SIZES for window_size in WINDOW_SIZES
]

conditions_grid_seq = [
    (
        data[("grid-seq", "GT", bin_size, "bin_vals")], 
        [(num_samples, data[("grid-seq", "sampled", bin_size, window_size, num_samples, "bin_vals")]) for num_samples in NUM_SAMPLES], 
        "grid-seq bin_size=" + str(bin_size//KBP) + "k window_size=" + str(window_size//KBP) + "k"
     ) 
     for bin_size in BIN_SIZES for window_size in WINDOW_SIZES
]

corr_as_func_of_samples(conditions_ice + conditions_grid_seq)


- ICE:
    - window size does not affect results
    - bin size does
- Grid-Seq:
    - actually reaches zero


## Plot some of the heatmaps for a visual verification

In [ ]:
DEFAULT_RANGE=(0, 501)
def plot_heatmap(datas, bg_color, x_range=DEFAULT_RANGE, y_range=DEFAULT_RANGE):
    fl = []
    for data, title in datas:
        if len(fl) == 0:
            f = figure(title=title)
        else:
            f = figure(title=title, x_range=fl[0].x_range, y_range=fl[0].y_range)
        d_filtered = {}
        for key, vals in data.items():
            d_filtered[key] = []
            for idx, v in enumerate(vals):
                if data["screen_left"][idx] >= x_range[0] and data["screen_right"][idx] < x_range[1] and \
                data["screen_bottom"][idx] >= y_range[0] and data["screen_top"][idx] < y_range[1]:
                    d_filtered[key].append(v)
        f.quad(
            left="screen_left",
            bottom="screen_bottom",
            right="screen_right",
            top="screen_top",
            fill_color="color",
            line_color=None,
            source=ColumnDataSource(data=d_filtered),
        )
        f.background_fill_color = bg_color
        f.grid.grid_line_color = None
        f.axis.ticker = []

        f.add_tools(
            HoverTool(
                tooltips=[
                    (
                        "(x, y)",
                        "(@chr_x @index_left .. @index_right, @chr_y @index_bottom .. @index_top)",
                    ),
                    ("score", "@score_total"),
                    ("color", "@color"),
                    ("reads by group", "A: @score_a, B: @score_b"),
                ]
            )
        )
        f.output_backend = OUTPUT_BACKEND
        fl.append(f)
    show(column(fl), notebook_handle=True)

plot_heatmap([
    (data[("ICE", "cooler", BIN_SIZES[0], "heatmap")], "cooler"), 
    (data[("ICE", "global", BIN_SIZES[0], WINDOW_SIZES[0], NUM_SAMPLES[-1], "heatmap")], "ice - num samples = " + str(NUM_SAMPLES[-1])),
    (data[("ICE", "global", BIN_SIZES[0], WINDOW_SIZES[0], NUM_SAMPLES[0], "heatmap")], "ice - num samples = " + str(NUM_SAMPLES[0])),
    (data[("raw-mirror", BIN_SIZES[0], "heatmap")], "raw data"),
    ], quarry.get_background_color(lib_sps_print))

plot_heatmap([
    (data[("grid-seq", "GT", BIN_SIZES[0], "heatmap")], "GT"), 
    (data[("grid-seq", "sampled", BIN_SIZES[0], WINDOW_SIZES[0], NUM_SAMPLES[-1], "heatmap")], "grid-seq = " + str(NUM_SAMPLES[-1])),
    (data[("grid-seq", "sampled", BIN_SIZES[0], WINDOW_SIZES[0], NUM_SAMPLES[0], "heatmap")], "grid-seq = " + str(NUM_SAMPLES[0])),
    (data[("raw-all", BIN_SIZES[0], "heatmap")], "raw data"),
    ], quarry.get_background_color(lib_sps_print))